##### <p> Samuel Wolfe <br> November 718, 2023 <br> MSBA 207 <br> Chapter 14 </p>

In [54]:
# need to run "pip install mlxtend" first
# need to run "pip install surprise" first
# "conda install -c conda-forge scikit-surprise" in "terminal" (Mac) or "Anaconda Prompt" (Windows)
%matplotlib inline
from IPython.display import clear_output

from pathlib import Path

import heapq
from collections import defaultdict

import pandas as pd
import matplotlib.pylab as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
pd.set_option('display.width', 250)

In [55]:
# Working directory:
#
# We assume that data are kept in the same directory as the notebook. If you keep your 
# data in a different folder, replace the argument of the `Path`
DATA = Path('E:/Aliit/School/MSBA/206/MSBA-206/dmba')
#DATA = Path('C:/Users/Min Li/OneDrive/teaching/DS110/dmba')
# and then load data using 
#
# pd.read_csv(DATA / ‘filename.csv’)
# Load and preprocess data set 

## Chapter 14 Problem 2

In [56]:
df_topics = pd.read_csv(DATA / 'CourseTopics.csv',dtype='bool')
#Removing spaces from column names
names = df_topics.columns.to_list()
for x in names:
    df_topics = df_topics.rename(columns={x:x.replace(' ','_')})
df_topics

,Intro,DataMining,Survey,Cat_Data,Regression,Forecast,DOE,SW
0,True,True,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False
2,False,True,False,True,True,False,False,True
3,True,False,False,False,False,False,False,False
4,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
360,False,False,False,True,False,False,False,False
361,False,True,False,True,False,False,False,True
362,False,False,False,False,False,False,False,True
363,False,False,False,True,False,False,False,False


In [57]:
#This section takes 56.9s
0.02
0.55

mini = 1
maxa = 40
limit1 = 100
limit2 = 100
cv1 = range(mini,maxa)
cv2 = range(1,limit2)
results = pd.DataFrame(columns=['min_support','min_threshold','lift'])
for x1 in cv1:
    clear_output(wait=True)
    print(x1)
    z1 = x1/limit1
    # create frequent itemsets
    itemsets = apriori(df_topics, use_colnames=True, min_support=z1)
    if itemsets.shape[0] == 0:
        break
    for x2 in cv2:
        z2 = x2/limit2
        # and convert into rules
        rules = association_rules(itemsets, metric='confidence', min_threshold=z2)
        try:
            limit_break = rules.sort_values(by=['lift'], ascending=False)['lift'].mean()
            results.loc[len(results.index)] = [z1,z2,limit_break]
        except:
            continue
results = results.sort_values(by='lift', ascending=False)
results

39


,min_support,min_threshold,lift
70,0.01,0.71,3.038238
62,0.01,0.63,3.020972
73,0.01,0.74,3.015365
72,0.01,0.73,3.015365
71,0.01,0.72,3.015365
...,...,...,...
3856,0.39,0.95,NaN
3857,0.39,0.96,NaN
3858,0.39,0.97,NaN
3859,0.39,0.98,NaN


#### I experimented a lot creating my own CV set there. quickly found out that the max mean lift we can get is about 91.25. So I am setting my values to 0.002 and 0.982.

In [58]:
# create frequent itemsets
itemsets = apriori(df_topics, use_colnames=True, min_support=0.002)

# and convert into rules
rules = association_rules(itemsets, metric='confidence', min_threshold=0.982)
print(rules.shape)
rules.sort_values(by=['support'], ascending=False).head(6)

(177, 10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
5,"(SW, Regression, DOE)",(Intro),0.019178,0.394521,0.019178,1.0,2.534722,0.011612,inf,0.617318
4,"(Cat_Data, Regression, DOE)",(Intro),0.010959,0.394521,0.010959,1.0,2.534722,0.006635,inf,0.612188
11,"(SW, Survey, Forecast)",(Cat_Data),0.008219,0.208219,0.008219,1.0,4.802632,0.006508,inf,0.798343
3,"(SW, Survey, Forecast)",(Intro),0.008219,0.394521,0.008219,1.0,2.534722,0.004977,inf,0.610497
6,"(Survey, DOE, DataMining)",(Cat_Data),0.008219,0.208219,0.008219,1.0,4.802632,0.006508,inf,0.798343
45,"(SW, Survey, Forecast)","(Cat_Data, Intro)",0.008219,0.071233,0.008219,1.0,14.038462,0.007634,inf,0.936464


In [59]:
sup_mean = rules.describe()['support'][1]
rules_best_sup = rules.loc[rules['support'] >= sup_mean].sort_values(by=['lift'], ascending=False )
rules_best_sup.head(6)

c:\Program Files\Python312\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Program Files\Python312\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\Sam\AppData\Local\Temp\ipykernel_14204\2504041522.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sup_mean = rules.describe()['support'][1]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
50,"(SW, Survey, Regression)","(DOE, Intro)",0.005479,0.046575,0.005479,1.0,21.470588,0.005224,inf,0.958678
19,"(Survey, Regression, Forecast)","(Intro, DataMining)",0.005479,0.054795,0.005479,1.0,18.250000,0.005179,inf,0.950413
26,"(Regression, DOE, DataMining)","(Cat_Data, Intro)",0.005479,0.071233,0.005479,1.0,14.038462,0.005089,inf,0.933884
45,"(SW, Survey, Forecast)","(Cat_Data, Intro)",0.008219,0.071233,0.008219,1.0,14.038462,0.007634,inf,0.936464
16,"(Intro, Survey, Regression, DataMining)",(Forecast),0.005479,0.139726,0.005479,1.0,7.156863,0.004714,inf,0.865014
12,"(SW, Survey, Regression)",(DOE),0.005479,0.172603,0.005479,1.0,5.793651,0.004534,inf,0.831956


#### Looking at the results, I initially was interested in finding the greatest mean lift, as that feels like the best place to start. So I set my CV to use lift to give my model the best chances. Then I filtered by the mean support as a cutoff, as the book indicates that a any variable that shows promise but has a low support is a lot less desirable. Then I sorted those by raw lift. Here we can see that if a student signs up for Regression, SW, Survey, they are very likely to sign up for Intro and DOE.

## Chapter 14 Problem 4

In [60]:
df_cosmetics = pd.read_csv(DATA / 'Cosmetics.csv', index_col='Trans.')
df_cosmetics = df_cosmetics.astype(dtype='bool')
#Removing spaces from column names
names = df_cosmetics.columns.to_list()
for x in names:
    df_cosmetics = df_cosmetics.rename(columns={x:x.replace(' ','_')})
df_cosmetics.head(12)

,Bag,Blush,Nail_Polish,Brushes,Concealer,Eyebrow_Pencils,Bronzer,Lip_liner,Mascara,Eye_shadow,Foundation,Lip_Gloss,Lipstick,Eyeliner
Trans.,,,,,,,,,,,,,,
1,False,True,True,True,True,False,True,True,True,False,False,False,False,True
2,False,False,True,False,True,False,True,True,False,False,True,True,False,False
3,False,True,False,False,True,True,True,True,True,True,True,True,True,False
4,False,False,True,True,True,False,True,False,False,False,True,False,False,True
5,False,True,False,False,True,False,True,True,True,True,False,True,True,False
6,False,False,False,False,True,False,False,False,False,False,False,False,False,True
7,False,True,True,True,True,False,True,True,True,True,True,True,False,False
8,False,False,True,True,False,False,True,False,True,True,False,True,False,False
9,False,False,False,False,True,False,False,False,False,False,True,False,True,False


#### 14.a These rows show purchases. For Trans. 1, it shows the customer purchased Blush, Nail_Polish, Brushes, Concealer, Bronzer, Lip_liner, Mascara, and Eyeliner. Broad strokes we can see that very few customers purchase Bag or Eyebrow_Pencils.

##### b. Consider the results of the association rules analysis shown in Table 14.16.
##### i. For the first row, explain the “confidence” output and how it is calculated.
##### ii. For the first row, explain the “support” output and how it is calculated.
##### iii. For the first row, explain the “lift” and how it is calculated.
##### iv. For the first row, explain the rule that is represented there in words.

                                      lhs               |         rhs        | support  |  confidence  |     lift

     1 {Blush, Concealer, Mascara, Eye.shadow, Lipstick} => {Eyebrow.Pencils}   0.013      0.3023255814   7.198228128

#### 14.b.i Confidence is the Probability of the antecedent and consequent divided by the Probability of the antecedent. Or the estimated  conditional probability that the items selected will lead to the other item being added. Looking at the first row of Table 14.16 in the book the confidence is 0.3023255814. This indicates it has a moderately low probability of being valid.
#### 14.b.ii Support is the Probability of the antecedent and consequent. Or the probability that if the antecedent is picked then the consequents being picked. Looking at the first row we see a support of 0.013. This means that if we look at the lhs variables in a database of transactions, there is a 0.013 chance the rhs value will be in the cart as well.
#### 14.b.iii Lift is the confidence over benchmark confidence. Lift greater than 1.0 suggests that there is some usefulness to the rule. A lift of 7.198228128. This indicates that there is a strong indication that the variables are associated.
#### 14.b.iiv Given a cart that has the lhs values, there is a 0.013 probability that the rhs values will be in the cart. This chance has a confidence of 0.3023255814, which is moderately low. The lift value though suggests that there is some value to this rule, as it indicates that the lhs values and the rhs values are at least marginally associated.

In [61]:
# create frequent itemsets
itemsets = apriori(df_cosmetics, use_colnames=True, min_support=0.1)

# and convert into rules
rules = association_rules(itemsets)
rules.head(3).drop(columns=['antecedent support','consequent support','conviction','zhangs_metric'])

,antecedents,consequents,support,confidence,lift,leverage
0,(Brushes),(Nail_Polish),0.149,1.00000,3.571429,0.107280
1,(Mascara),(Eye_shadow),0.321,0.89916,2.359999,0.184983
2,(Eye_shadow),(Mascara),0.321,0.84252,2.359999,0.184983


#### 14.c.i Looking at the first three rules by index value.
#### If a cart has Brushes then there is a 14.9% chance Nail_Polish will be in the cart as well. The confidence of these is 1, tell us this is always true. The lift of 3.571429 indicates there is some value to this rule being applied. With a leverage of 0.107280 we learn that these two items are fairly independent.  
#### If a cart has Eye_shadow there is a 32.1% chance Mascara will be in the cart as well. The confidence of this is 0.84252, which tells us this rule is mostly always true. A lift of 2.359999 indicates this rule has value. The leverage of 0.184983 tells us these two items are a bit more associated than in index 0.
#### If a cart has Mascara there is a 32.1% chance Eye_shadow will be in the cart as well. The confidence of this is 0.0.89916, which tells us this rule is mostly always true. A lift of 2.359999 indicates this rule has value. The leverage of 0.184983 tells us these two items are a bit more associated than in index 0.
#### The interesting part with index 1 & 2 is that these items are essentially mirrored. This shows us its better to suggest Eye_shadow to a cart with Mascara than Mascara to a cart with Eye_shadow.

In [62]:
rules.head(24).drop(columns=['antecedent support','consequent support','conviction','zhangs_metric'])

,antecedents,consequents,support,confidence,lift,leverage
0,(Brushes),(Nail_Polish),0.149,1.000000,3.571429,0.107280
1,(Mascara),(Eye_shadow),0.321,0.899160,2.359999,0.184983
2,(Eye_shadow),(Mascara),0.321,0.842520,2.359999,0.184983
3,"(Blush, Lip_liner)",(Concealer),0.108,0.870968,1.970515,0.053192
4,"(Mascara, Blush)",(Eye_shadow),0.169,0.918478,2.410704,0.098896
5,"(Blush, Eye_shadow)",(Mascara),0.169,0.928571,2.601040,0.104026
6,"(Mascara, Nail_Polish)",(Eye_shadow),0.119,0.888060,2.330865,0.067946
7,"(Nail_Polish, Eye_shadow)",(Mascara),0.119,0.908397,2.544529,0.072233
8,"(Lip_liner, Bronzer)",(Concealer),0.103,0.804687,1.820560,0.046424
9,"(Eyeliner, Bronzer)",(Concealer),0.119,0.815068,1.844046,0.054468


#### 14.c.ii Looking at the first 24 by index, It might be better to reduce the number of matching pairs. We have a lot 3 item pairings that are all very likely. It would take a bit of effort, but I would likely remove but 1 rule for each set of pairings, keeping the one with the highest confidence.